In [117]:
import pandas as pd
from smith_waterman import algs
import numpy as np
pd.options.mode.chained_assignment = None

Outline/main function of my genetic algorithm. Followed a structure adapted from https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3

Details on the interior functions are found below. 

In [140]:
def GA(alignments):
    '''
    Input: matrix of sequences aligned and results of alignment, see 'data' below 
    Output: the best scoring matrix learned by the algorithm
    '''
    old_best = 0
    pop = initialize_population()
    fitness = compute_fitness(pop, alignments)
    best = max(fitness.values())
    print('best: ', best)
    while best - old_best > 0.0001: 
        parents = selection(fitness)
        pop = crossover(parents, pop)
        pop = mutate(pop)
        fitness = compute_fitness(pop, alignments)
        old_best = best
        best = max(fitness.values())
        loc = max(fitness, key=fitness.get)
        best_mat = pop[loc]
    print('pre_best:', best)
    print('new_best: ', best)
    
    return best_mat

Initializing the population:
Started with PAM100 as the base and create three additional matrices with 50 positions mutated from their PAM100 state

In [ ]:
def initialize_population():
    p100 = algs.read_scoring_matrix("PAM100")
    pop = dict.fromkeys(range(4))
    pop[0] = p100.copy()
    pop[1] = p100.copy()
    pop[2] = p100.copy()
    pop[3] = p100.copy()
    for i in range(50):
        pop[1] = one_mutation(pop[1])
        pop[2] = one_mutation(pop[2])
        pop[3] = one_mutation(pop[3])
        
    return pop

Computing the fitness of each matrix in a population:
Used AUC of the scoring result of each matrix as the fitness score. 

Something here is not working quite right and the new scores being added are very incorrect but I have not been able to track down how that error is coming about. If this were fixed I believe my algorithm would perform well

In [ ]:
def compute_fitness(pop, alignments):
    '''
    Input: dict of population matrices, dataframe of alignments for each sequence pair
    Output: dict of fitness scores for each matrix in the population
    '''
    fitness = dict.fromkeys(range(4))
    for i in pop.keys():
        a = alignments.copy(deep=True)
        for row in a.index:
            '''            
            this comes form a function from my algs file, it works in normal testing which leads me to believe there it a bug in my inputs that I'm not catch
            '''            
            a['New_Score'].iloc[row] = algs.get_score(a['a1'].iloc[row], a['a2'].iloc[row], a['a'].iloc[row], pop[i], 'no')
        auc = roc_auc(a)
        fitness[i] = auc
    return fitness

def roc_auc(df):
    '''
    Input: a dataframe of sequence pairs and alignments to calculate the AUC performance
    Output: AUC
    '''
    df = df.sort_values(by='New_Score', ascending=False)
    TPs = []
    FPs = []
    t = 0
    f = 0
    for row in df.index:
        if df['Truth'].iloc[row] == 1:
            t += 1
        else:
            f += 1
        TPs.append(t/50)
        FPs.append(f/50)
        
    auc = round(np.trapz(TPs, FPs), 3)
    
    return auc

Crossing over and mutation to generate a next generation population:
Crossing over is done by taking a random chunk off of each parent population and re-appending it to the other. This took a bit of work to get working properly with numpy but I'm pretty proud of how it turned out. 


//

Mutation is done 10 times per matrix each population in random locations to random values. There is probably a better way to randomize how many mutations happen per matrix but I haven't had time to implement that yet. 



In [127]:
def crossover(parents, pop):
    p1 = np.triu(pop[parents[0]].to_numpy())
    p2 = np.triu(pop[parents[1]].to_numpy())
    names = pop[parents[0]].columns
    
    split = int(np.random.randint(low = 0, high = 23, size=1))
    p1_1 = p1[:,:split]
    p1_2 = p1[:,split:]
    p2_1 = p2[:,:split]
    p2_2 = p2[:,split:]
    
    c1 = np.append(p1_1, p2_2, 1)
    c2 = np.append(p2_1, p1_2, 1)
    
    c1 = pd.DataFrame(c1 + c1.T - np.diag(np.diag(c1)), columns = names, index = names)
    c2 = pd.DataFrame(c2 + c2.T - np.diag(np.diag(c2)), columns = names, index = names)
    
    new_pop = dict.fromkeys(range(4))
    new_pop[0] = pop[parents[0]]
    new_pop[1] = pop[parents[1]]
    new_pop[2] = c1
    new_pop[3] = c2
    return new_pop
def one_mutation(mat):
    i = int(np.random.randint(low = 0, high = 23, size=1))
    j = int(np.random.randint(low = 0, high = 23, size=1))

    new = int(np.random.randint(low = -10, high = 10, size=1))
    mat.iloc[i][j] = new
    mat.iloc[j][i] = new
    
    return mat

def mutate(pop):
    for mat in pop:
        for mut in range(10):
            pop[mat] = one_mutation(pop[mat])
    return pop 

Selection:
    choose two matrices with the highest fitness and return their dict keys for reference in generating offspring populations

In [119]:

def selection(fitness):
    m1 = max(fitness.values())
    m1loc = max(fitness, key=fitness.get)
    max(fitness, key=fitness.get)
    m2 = 0
    for i in fitness.keys():
        if fitness[i] > m2 and fitness[i] < m1:
            m2 = fitness[i]
    m2loc = list(fitness.keys())[list(fitness.values()).index(m2)]
    
    return m1loc, m2loc

Copied in from algs to use here and tweak for adding the info needed here (old version didn't save alignments, just scores)

In [141]:
def all_sw(d, method, o, e):
    for row in d.index:
        a = algs.read_seq_file(d['A'].iloc[row])
        b = algs.read_seq_file(d['B'].iloc[row])
        s, align1, align2, align = algs.sw(a, b, method, o, e)
        d['PAM100_Score'].iloc[row] += s
        d['a1'].iloc[row] = align1
        d['a2'].iloc[row] = align2
        d['a'].iloc[row] = align
    return d

def get_data():
    p = pd.read_table('Pospairs.txt', header = None, sep = ' ', names = ['A','B'])
    n = pd.read_table('Negpairs.txt', header = None, sep = ' ', names = ['A', 'B'])
    p['Truth'] = 1
    n['Truth'] = 0
    pairs = p.append(n, ignore_index=True)
    data = pairs
    data["PAM100_Score"] = 0
    data['a1']=''
    data['a2']=''
    data['a']=''
    data['New_Score'] = ''
    return data


Below is some of my random attempts to figure out what is wrong with getting the scores. Feel free to ignore

In [137]:
algs.get_score(data['a1'].iloc[0], data['a2'].iloc[0], data['a'].iloc[0], "PAM100")

200

In [138]:
data['a1'].iloc[0]

'L--DSARFRYVLSEEFDAPVQNVEGTILGEHGDAQVPVFSKVRVDGTDPEFSGDEKEQLLGDLQESAMDVIERKGATEWGPARGVAHMVEAILHDTGEVLPASVKLEGEFGHEDTAFGVPVRLGSNGVEEIVEWDLDDYEQDLMADAAEKL'

In [139]:
data

,A,B,Truth,PAM100_Score,a1,a2,a,New_Score
47,sequences/prot-0776.fa,sequences/prot-0779.fa,1,242,L--DSARFRYVLSEEFDAPVQNVEGTILGEHGDAQVPVFSKVRVDG...,TILDTARFRFLLGEYFSVAPQNVHAYIIGEHGDTELPVWSQAYIG-...,.--D.ARFR..L.E.F....QNV...I.GEHGD...PV.S.....-...,
44,sequences/prot-0614.fa,sequences/prot-0616.fa,1,88,K-VAVLGAAGGIGQALALLLKTQLPSG-SELSLYDIAP--VT-PGV...,TKVSVVGAAGTVGAAA-GY-NIALRDIADEVVFVDIPDKEDDTVGQ...,.-V.V.GAAG..G.A.-..-...L...-.E....DI..--..-.G....,
14,sequences/prot-0326.fa,sequences/prot-0327.fa,1,75,IDVLLGADDGSLAFVPSEFSISPGEKIVFKNNAGFPHNIVFDEDSI...,VKMLNSGPGGMMVFDPALVRLKPGDSIKFL-----PTDKGHNVETI...,...L.....G...F.P......PG..I.F.-----P.........I...,
46,sequences/prot-0735.fa,sequences/prot-0736.fa,1,54,D-----------L-ALY------EL-----ALLRFAMDFMARRGFL...,DQEVRYRQRYLDLIANDKSRQTFVVRSKILAAIRQ---FMVARGFM...,D-----------L-A..------..-----A..R.---FM..RGF....,
4,sequences/prot-0077.fa,sequences/prot-0082.fa,1,53,GN-GT--------IDFPEFLSLMARKMKEQDSEE-ELIEAFKVFDR...,GLNDFQKQKIKFTFDFF--LDM-NHDGSIQDNDFEDMMTRYKE--V...,G.-..--------.DF.--L..-......QD...-......K.--....,
...,...,...,...,...,...,...,...,...
57,sequences/prot-0148.fa,sequences/prot-0369.fa,0,18,RM----------K--Q-LEDKVEE-----L--LSKNYHLE------...,QDYSFCLFSPFVYYKCDLEVTLSPHTSGAHGLLVRWCPTGTPTKPT...,..----------.--.-LE.....-----.--L.......------...,
58,sequences/prot-0166.fa,sequences/prot-0839.fa,0,17,QF-H----WK-N---SNQIK--IL-----GN--------QG---SF...,DCYHGDGQSYRGTSSTTTTGKKCQSWSSMTPHRHQKTPENYPNAGL...,..-H----..-.---.....--..-----..--------..---.....,
61,sequences/prot-0228.fa,sequences/prot-0121.fa,0,17,I--SRG--IDK-----QGA----L-LL-EQDGIIKP-W--M---GG...,TPTGRTLATAVMRKHRLAERLLTDIIGLDINKVHDEACRWEHVMSD...,.--.R.--...-----...----.-..-........-.--.---.....,
90,sequences/prot-0750.fa,sequences/prot-0875.fa,0,15,Q-QDPEGNK,ZHADPICNK,.-.DP..NK,
